# Notebook as a Step using Notebooks Executor
The following sample shows how to use the notebook executor as part of a Vertex AI


## Libraries and Variables

In [ ]:
!pip install --upgrade kfp
!pip install --upgrade google-cloud-aiplatform
!pip install --upgrade google-cloud-pipeline-components

In [ ]:
import kfp
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from datetime import datetime
from kfp.dsl.types import GCPProjectID

In [ ]:
kfp.__version__

In [ ]:
# Variables
PROJECT_ID = 'mam-nooage'
REGION = 'us-central1' # This is where the KMS and Metastore resides from the project configuration

ROOT_PATH = ".."
PIPELINE_ROOT_PATH = f'gs://{PROJECT_ID}-vertex-root'
PACKAGE_PATH = 'notebook-as-a-step-sample-pipeline.json'

COMPONENT_YAML_PATH = f'{ROOT_PATH}/component.yaml'
WORKING_BUCKET_NAME = f'{PROJECT_ID}-naas'
INPUT_NOTEBOOK_FILE = f'gs://{WORKING_BUCKET_NAME}/runnables/run_create_bucket.ipynb'
OUTPUT_NOTEBOOK_FOLDER = f'gs://{WORKING_BUCKET_NAME}/outputs'

INPUT_NOTEBOOK_FILE

In [ ]:
!gsutil ls "{PIPELINE_ROOT_PATH}" || gsutil mb -l "{REGION}" "{PIPELINE_ROOT_PATH}"
!gsutil cp -r "{ROOT_PATH}/samples/runnables/*" "gs://{WORKING_BUCKET_NAME}/runnables"
!gsutil ls "gs://{WORKING_BUCKET_NAME}/runnables"

## Prerequisites
You need to configure your project as detailed in https://cloud.google.com/vertex-ai/docs/pipelines/configure-project

In [ ]:
@kfp.dsl.pipeline(
    name="notebook-as-a-step-sample",
    pipeline_root=PIPELINE_ROOT_PATH)
def pipeline(
        project_id: str, 
        execution_id: str, 
        input_notebook:str, 
        output_notebook_folder:str):
    
    execute_notebook_component = kfp.components.load_component_from_file(COMPONENT_YAML_PATH)
    execute_notebook_op = execute_notebook_component(
        project_id=project_id,
        execution_id=execution_id,
        input_notebook_file=input_notebook,
        output_notebook_folder=output_notebook_folder,
        block='true',
        fail_pipeline=False,
        parameters=f'PROJECT_ID={project_id},EXECUTION_ID={execution_id}'
    )

For additional details about:
- Google Cloud Notebook Executor template, see the [ExecutionTemplate API documentation](https://cloud.google.com/notebooks/docs/reference/rest/v1/ExecutionTemplate).
- Pipeline types, see [kfp.dsl.types](https://kubeflow-pipelines.readthedocs.io/en/latest/_modules/kfp/dsl/types.html).
- Notebook parameters, see [Papermill parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html)

## Run pipeline

In [ ]:
NOW = datetime.now().strftime("%Y%m%d_%H%M%S")
EXECUTION_ID=f'naas_{NOW}'

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=PACKAGE_PATH)

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION)

response = api_client.create_run_from_job_spec(
    PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT_PATH,
    parameter_values={
        'project_id': PROJECT_ID,
        'execution_id': EXECUTION_ID,
        'input_notebook': INPUT_NOTEBOOK_FILE,
        'output_notebook_folder': OUTPUT_NOTEBOOK_FOLDER
    })

In [ ]:
!gsutil cp gs://mam-hub-naas/outputs/run_create_bucket_at_2021_09_13_07_16_14.ipynb .